### Imports

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import sqlite3

Create a sql connection from which to read tables

In [24]:
con = sqlite3.connect("data/database.sqlite")
cursor = con.cursor()

#### WHEN TO USE CURSOR EXECUTE VS PD.READ_SQL?????

### Prelimanary Analysis

Read Tables

In [25]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", con)
print(tables)

    type               name           tbl_name  rootpage  \
0  table    sqlite_sequence    sqlite_sequence         4   
1  table  Player_Attributes  Player_Attributes        11   
2  table             Player             Player        14   
3  table              Match              Match        18   
4  table             League             League        24   
5  table            Country            Country        26   
6  table               Team               Team        29   
7  table    Team_Attributes    Team_Attributes         2   

                                                 sql  
0             CREATE TABLE sqlite_sequence(name,seq)  
1  CREATE TABLE "Player_Attributes" (\n\t`id`\tIN...  
2  CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...  
3  CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...  
4  CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...  
5  CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...  
6  CREATE TABLE "Team" (\n\t`id`\tINTEGER PRIMARY...  
7  CREATE TABLE `Te

Print all tables in a for loop:

In [26]:
for table_name in tables['name']:
    print(table_name.upper(), pd.read_sql_query("SELECT COUNT(*) FROM %s" % table_name, con), "\n", pd.read_sql_query("SELECT * from %s LIMIT 5" % table_name, con), "\n\n\n")

SQLITE_SEQUENCE    COUNT(*)
0         7 
       name     seq
0     Team  103916
1  Country   51958
2   League   51958
3    Match   51958
4   Player   11075 



PLAYER_ATTRIBUTES    COUNT(*)
0    183978 
    id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00              67   
1   2              218353         505942  2015-11-19 00:00:00              67   
2   3              218353         505942  2015-09-21 00:00:00              62   
3   4              218353         505942  2015-03-20 00:00:00              61   
4   5              218353         505942  2007-02-22 00:00:00              61   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0         71          right              medium              medium        49   
1         71          right              medium              medium        49   
2         66          right              medium              mediu

# OBJECTIVES

Average Number of Goals scored each team
Average Number of goals conceded each team
^^^ Same as above but home vs away advantage
^^^ Same as above but for overall league team is in (which league more competitive?)
Average Stats of players in team
Min-Max player stats per team (everyone is average? Few excellent players and few bad players same team?

Count most used formations
Average goals scored and conceded per formation

Assign formation to team and see if stats match up of formation vs team performance using formation.

Impactful substitutions and/or starters? (harry winks starts, game is gone)


Dump all the tables into separate csv files to make life easier

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, con)
    table.to_csv('data/' + table_name + '.csv', index_label='index')
cursor.close()
con.close()

In [6]:
print(tables)

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


Import all newly created csv files/tables

In [7]:
### We created the variable "tables" 
### from where the csv naming convention for the dump came from
### So it can be looped to import all the tables
for table_name in tables:
    print(str(table_name[0]) + '.csv')

sqlite_sequence.csv
Player_Attributes.csv
Player.csv
Match.csv
League.csv
Country.csv
Team.csv
Team_Attributes.csv


In [70]:
Player_Attributes = pd.read_csv('data/Player_Attributes.csv')
Player = pd.read_csv('data/Player.csv')
Match = pd.read_csv('data/Match.csv')
League = pd.read_csv('data/League.csv')
Country = pd.read_csv('data/Country.csv')
Team = pd.read_csv('data/Team.csv')
Team_Attributes = pd.read_csv('data/Team_Attributes.csv')

In [71]:
Player_Attributes.shape, Player.shape, Match.shape, League.shape, Country.shape, Team.shape, Team_Attributes.shape

((183978, 43),
 (11060, 8),
 (25979, 116),
 (11, 4),
 (11, 3),
 (299, 6),
 (1458, 26))

In [72]:
Country.head(5)

,index,id,name
0,0,1,Belgium
1,1,1729,England
2,2,4769,France
3,3,7809,Germany
4,4,10257,Italy


In [73]:
Player_Attributes.drop(['index'], axis=1, inplace=True)
Player.drop(['index'], axis=1, inplace=True)
Match.drop(['index'], axis=1, inplace=True)
League.drop(['index'], axis=1, inplace=True)
Country.drop(['index'], axis=1, inplace=True)
Team.drop(['index'], axis=1, inplace=True)
Team_Attributes.drop(['index'], axis=1, inplace=True)

In [74]:
Country.head(2)

,id,name
0,1,Belgium
1,1729,England


In [75]:
Player.head(2)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146


In [76]:
Player_Attributes.columns, Player.columns, Match.columns, League.columns, Country.columns, Team.columns, Team_Attributes.columns

(Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
        'potential', 'preferred_foot', 'attacking_work_rate',
        'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
        'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
        'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
        'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
        'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
        'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
        'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
        'gk_reflexes'],
       dtype='object'),
 Index(['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'birthday',
        'height', 'weight'],
       dtype='object'),
 Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
        'match_api_id', 'home_team_api_id', 'away_team_api_id',
        'hom

In [77]:
Player_Attributes.head(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0


In [78]:
Team_Attributes.head(3)

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover


In [79]:
Team.head(3)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL


In [80]:
Player.head(3)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163


In [81]:
Match.head(3)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.5,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75


In [82]:
Country.head(3)

,id,name
0,1,Belgium
1,1729,England
2,4769,France


In [83]:
League.head(3)

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1


Players dont have team id's linking them to their respective teams?
Does this mean I have to drop the player aspect of the dataset entirely?

Merge All Tables into Match Table on IDs

In [84]:
Match_Merged_df = pd.merge(Match, Country, how='outer', left_on= 'country_id', right_on='id')

In [85]:
Match_Merged_df = pd.merge(Match, League, how='outer', left_on= 'league_id', right_on='id')

In [86]:
Match_Merged_df.columns

Index(['id_x', 'country_id_x', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'id_y', 'country_id_y',
       'name'],
      dtype='object', length=118)

In [87]:
Match_Merged_df.head(2)

,id_x,country_id_x,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCA,GBH,GBD,GBA,BSH,BSD,BSA,id_y,country_id_y,name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.50,1.78,3.25,4.00,1.73,3.40,4.2,1,1,Belgium Jupiler League
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.25,1.85,3.25,3.75,1.91,3.25,3.6,1,1,Belgium Jupiler League
